In [2]:
import os
from openai import AzureOpenAI


In [ ]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment="o3-mini",
    azure_api_version="2023-05-15",
    azure_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

TypeError: AzureOpenAI.__init__() got an unexpected keyword argument 'azure_api_version'